In [1]:
import pandas as pd
import numpy as np

In [2]:
from sklearn.impute import SimpleImputer

# Define constants

In [3]:
%store -r F_1_TRAIN_INTERM
%store -r F_1_TRAIN_ENCODED
%store -r F_1_TEST_INTERM
%store -r F_1_TEST_ENCODED
%store -r DATA_INPUT_DIR
%store -r DATA_OUTPUT_DIR

# Read Train Data To Dataframe

In [4]:
df = pd.read_csv(f"{DATA_OUTPUT_DIR}/{F_1_TRAIN_INTERM}", sep='|', index_col=0, dtype={"CustomerID": 'object'})
df_test = pd.read_csv(f"{DATA_OUTPUT_DIR}/{F_1_TEST_INTERM}", sep='|', index_col=0, dtype={"CustomerID": 'object'})

In [5]:
df.shape

(80000, 12)

In [6]:
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,randint0,randint1,randint2,randint3
75220,542544,17165D,NaN,-10,1/28/2011 14:43,0.00,NaN,United Kingdom,2,2,105,1964
48955,540482,22470,HEART OF WICKER LARGE,6,1/7/2011 15:48,2.95,14672,United Kingdom,1,2,136,20035
44966,540247,22127,PARTY CONES CARNIVAL ASSORTED,18,1/5/2011 15:56,1.25,15464,United Kingdom,2,5,140,45537
13568,537434,22422,TOOTHPASTE TUBE PEN,3,12/6/2010 16:57,1.28,NaN,United Kingdom,1,8,133,21122
92727,544204,22173,METAL 4 HOOK HANGER FRENCH CHATEAU,1,2/17/2011 10:30,5.79,NaN,United Kingdom,1,1,106,84781


# Look for Missing Values

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 80000 entries, 75220 to 15795
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   InvoiceNo    80000 non-null  object 
 1   StockCode    80000 non-null  object 
 2   Description  79752 non-null  object 
 3   Quantity     80000 non-null  int64  
 4   InvoiceDate  80000 non-null  object 
 5   UnitPrice    80000 non-null  float64
 6   CustomerID   52017 non-null  object 
 7   Country      80000 non-null  object 
 8   randint0     80000 non-null  int64  
 9   randint1     80000 non-null  int64  
 10  randint2     80000 non-null  int64  
 11  randint3     80000 non-null  int64  
dtypes: float64(1), int64(5), object(6)
memory usage: 7.9+ MB


In [8]:
df.isnull().sum()

InvoiceNo          0
StockCode          0
Description      248
Quantity           0
InvoiceDate        0
UnitPrice          0
CustomerID     27983
Country            0
randint0           0
randint1           0
randint2           0
randint3           0
dtype: int64

In [9]:
df_test.isnull().sum()

InvoiceNo         0
StockCode         0
Description      51
Quantity          0
InvoiceDate       0
UnitPrice         0
CustomerID     6932
Country           0
randint0          0
randint1          0
randint2          0
randint3          0
dtype: int64

In [10]:
imputer1 = SimpleImputer(missing_values=np.nan, strategy='constant', fill_value='guest', copy=False, add_indicator=True)
imputer2 = SimpleImputer(missing_values=np.nan, strategy='constant', fill_value='N.A.', copy=False, add_indicator=True)

In [11]:
filled_cols1 =["CustomerID_filled", "is_CustomerID_null"]
CustomerID_filled = pd.DataFrame(imputer1.fit_transform(df[['CustomerID']]), columns=filled_cols1)
CustomerID_filled_test = pd.DataFrame(imputer1.fit_transform(df_test[['CustomerID']]), columns=filled_cols1)

filled_cols2 =["Description_filled", "is_Description_null"]
Description_filled = pd.DataFrame(imputer2.fit_transform(df[['Description']]), columns=filled_cols2)
Description_filled_test = pd.DataFrame(imputer2.fit_transform(df_test[['Description']]), columns=filled_cols2)

In [12]:
CustomerID_filled["is_CustomerID_null"] = CustomerID_filled["is_CustomerID_null"].map(int)
Description_filled["is_Description_null"] = Description_filled["is_Description_null"].map(int)

CustomerID_filled_test["is_CustomerID_null"] = CustomerID_filled_test["is_CustomerID_null"].map(int)
Description_filled_test["is_Description_null"] = Description_filled_test["is_Description_null"].map(int)

In [13]:
df_filled = pd.concat([df.drop(["CustomerID", "Description"], axis=1).reset_index(), CustomerID_filled, 
                       Description_filled], axis=1)
df_test_filled = pd.concat([df_test.drop(["CustomerID", "Description"], axis=1).reset_index(), CustomerID_filled_test,
                            Description_filled_test], axis=1)

In [14]:
df_filled.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80000 entries, 0 to 79999
Data columns (total 15 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   index                80000 non-null  int64  
 1   InvoiceNo            80000 non-null  object 
 2   StockCode            80000 non-null  object 
 3   Quantity             80000 non-null  int64  
 4   InvoiceDate          80000 non-null  object 
 5   UnitPrice            80000 non-null  float64
 6   Country              80000 non-null  object 
 7   randint0             80000 non-null  int64  
 8   randint1             80000 non-null  int64  
 9   randint2             80000 non-null  int64  
 10  randint3             80000 non-null  int64  
 11  CustomerID_filled    80000 non-null  object 
 12  is_CustomerID_null   80000 non-null  int64  
 13  Description_filled   80000 non-null  object 
 14  is_Description_null  80000 non-null  int64  
dtypes: float64(1), int64(8), object(6)
m

In [15]:
df_filled.isnull().sum(), df_test_filled.isnull().sum()

(index                  0
 InvoiceNo              0
 StockCode              0
 Quantity               0
 InvoiceDate            0
 UnitPrice              0
 Country                0
 randint0               0
 randint1               0
 randint2               0
 randint3               0
 CustomerID_filled      0
 is_CustomerID_null     0
 Description_filled     0
 is_Description_null    0
 dtype: int64,
 index                  0
 InvoiceNo              0
 StockCode              0
 Quantity               0
 InvoiceDate            0
 UnitPrice              0
 Country                0
 randint0               0
 randint1               0
 randint2               0
 randint3               0
 CustomerID_filled      0
 is_CustomerID_null     0
 Description_filled     0
 is_Description_null    0
 dtype: int64)

In [16]:
df_filled.set_index('index', inplace=True)
df_test_filled.set_index('index', inplace=True)

In [17]:
df_filled.head()

,InvoiceNo,StockCode,Quantity,InvoiceDate,UnitPrice,Country,randint0,randint1,randint2,randint3,CustomerID_filled,is_CustomerID_null,Description_filled,is_Description_null
index,,,,,,,,,,,,,,
75220,542544,17165D,-10,1/28/2011 14:43,0.00,United Kingdom,2,2,105,1964,guest,1,N.A.,1
48955,540482,22470,6,1/7/2011 15:48,2.95,United Kingdom,1,2,136,20035,14672,0,HEART OF WICKER LARGE,0
44966,540247,22127,18,1/5/2011 15:56,1.25,United Kingdom,2,5,140,45537,15464,0,PARTY CONES CARNIVAL ASSORTED,0
13568,537434,22422,3,12/6/2010 16:57,1.28,United Kingdom,1,8,133,21122,guest,1,TOOTHPASTE TUBE PEN,0
92727,544204,22173,1,2/17/2011 10:30,5.79,United Kingdom,1,1,106,84781,guest,1,METAL 4 HOOK HANGER FRENCH CHATEAU,0


# And now encoding non numerical data

In [18]:
df_filled.dtypes

InvoiceNo               object
StockCode               object
Quantity                 int64
InvoiceDate             object
UnitPrice              float64
Country                 object
randint0                 int64
randint1                 int64
randint2                 int64
randint3                 int64
CustomerID_filled       object
is_CustomerID_null       int64
Description_filled      object
is_Description_null      int64
dtype: object

In [19]:
df_filled['InvoiceYear'] = pd.to_datetime(df_filled["InvoiceDate"]).map(lambda d: d.year)
df_filled['Num_Words_Desc'] = df_filled["Description_filled"].map(lambda s: len(str(s).split(' ')))

df_test_filled['InvoiceYear'] = pd.to_datetime(df_test_filled["InvoiceDate"]).map(lambda d: d.year)
df_test_filled['Num_Words_Desc'] = df_test_filled["Description_filled"].map(lambda s: len(str(s).split(' ')))

In [20]:
df_filled.head()

,InvoiceNo,StockCode,Quantity,InvoiceDate,UnitPrice,Country,randint0,randint1,randint2,randint3,CustomerID_filled,is_CustomerID_null,Description_filled,is_Description_null,InvoiceYear,Num_Words_Desc
index,,,,,,,,,,,,,,,,
75220,542544,17165D,-10,1/28/2011 14:43,0.00,United Kingdom,2,2,105,1964,guest,1,N.A.,1,2011,1
48955,540482,22470,6,1/7/2011 15:48,2.95,United Kingdom,1,2,136,20035,14672,0,HEART OF WICKER LARGE,0,2011,4
44966,540247,22127,18,1/5/2011 15:56,1.25,United Kingdom,2,5,140,45537,15464,0,PARTY CONES CARNIVAL ASSORTED,0,2011,4
13568,537434,22422,3,12/6/2010 16:57,1.28,United Kingdom,1,8,133,21122,guest,1,TOOTHPASTE TUBE PEN,0,2010,3
92727,544204,22173,1,2/17/2011 10:30,5.79,United Kingdom,1,1,106,84781,guest,1,METAL 4 HOOK HANGER FRENCH CHATEAU,0,2011,6


In [21]:
df_test_filled.head()

,InvoiceNo,StockCode,Quantity,InvoiceDate,UnitPrice,Country,randint0,randint1,randint2,randint3,CustomerID_filled,is_CustomerID_null,Description_filled,is_Description_null,InvoiceYear,Num_Words_Desc
index,,,,,,,,,,,,,,,,
75721,542608,21873,1,1/30/2011 13:51,1.25,United Kingdom,1,6,121,36718,16770,0,IF YOU CAN'T STAND THE HEAT MUG,0,2011,7
80184,543022,22318,4,2/2/2011 14:37,2.95,United Kingdom,1,9,141,55893,16771,0,FIVE HEART HANGING DECORATION,0,2011,4
19864,537898,82494L,2,12/9/2010 10:42,2.95,United Kingdom,1,9,106,94022,17377,0,WOODEN FRAME ANTIQUE WHITE,0,2010,5
76699,542646,22840,1,1/31/2011 11:42,7.95,United Kingdom,2,5,117,79599,17220,0,ROUND CAKE TIN VINTAGE RED,0,2011,5
92991,544211,20972,12,2/17/2011 11:05,1.25,United Kingdom,1,7,122,69365,13018,0,PINK CREAM FELT CRAFT TRINKET BOX,0,2011,7


In [22]:
df_filled.dtypes

InvoiceNo               object
StockCode               object
Quantity                 int64
InvoiceDate             object
UnitPrice              float64
Country                 object
randint0                 int64
randint1                 int64
randint2                 int64
randint3                 int64
CustomerID_filled       object
is_CustomerID_null       int64
Description_filled      object
is_Description_null      int64
InvoiceYear              int64
Num_Words_Desc           int64
dtype: object

# Encode dummies with 5 max labels and 30% max dispersion

In [35]:
nb_dummy_max = 5
ratio_max = .01

In [36]:
df_encoded = df_filled
df_test_encoded = df_test_filled

In [37]:
def build_dummy_vals(df_arg, nb_dummy_max_arg, ratio_max_arg):
    dummy_val_dict = {}
    cols = [k for k,v in dict(df_arg.dtypes).items() if v.name in ['category', 'object']]
    for c in cols:
        vc = df_arg[c].value_counts()/sum(df_arg[c].value_counts()) > ratio_max_arg
        dummy_labels = [k for (k,v) in vc.items() if v][:nb_dummy_max_arg]
        dummy_val_dict[c] = dummy_labels
    return dummy_val_dict

In [38]:
def apply_dummy_vals(df_arg, dummy_dict):
    for c, dummy_labels in dummy_dict.items():
        tmp_enc = df_arg[c].map(lambda v: v if v in dummy_labels else 'other')
        dummies = pd.get_dummies(tmp_enc, prefix='dumm_'+c)
        df_arg = pd.concat([df_arg, dummies], axis=1)
    return df_arg

In [39]:
dummy_val_dict = build_dummy_vals(df_encoded, nb_dummy_max, ratio_max)

In [40]:
dummy_val_dict

{'InvoiceNo': [],
 'StockCode': [],
 'InvoiceDate': [],
 'Country': ['United Kingdom', 'Germany', 'France', 'EIRE'],
 'CustomerID_filled': ['guest'],
 'Description_filled': []}

In [41]:
df_encoded = apply_dummy_vals(df_encoded, dummy_val_dict)
df_test_encoded = apply_dummy_vals(df_test_encoded, dummy_val_dict)

In [42]:
cols_todrop = [k for k,v in dict(df_encoded.dtypes).items() if v == np.dtype('O')]

In [43]:
cols_todrop

['InvoiceNo',
 'StockCode',
 'InvoiceDate',
 'Country',
 'CustomerID_filled',
 'Description_filled']

In [44]:
df_encoded.drop(cols_todrop, axis=1, inplace=True)
df_test_encoded.drop(cols_todrop, axis=1, inplace=True)

In [45]:
df_encoded

,Quantity,UnitPrice,randint0,randint1,randint2,randint3,is_CustomerID_null,is_Description_null,InvoiceYear,Num_Words_Desc,...,dumm_StockCode_other,dumm_InvoiceDate_other,dumm_Country_EIRE,dumm_Country_France,dumm_Country_Germany,dumm_Country_United Kingdom,dumm_Country_other,dumm_CustomerID_filled_guest,dumm_CustomerID_filled_other,dumm_Description_filled_other
index,,,,,,,,,,,,,,,,,,,,,
75220,-10,0.00,2,2,105,1964,1,1,2011,1,...,1,1,0,0,0,1,0,1,0,1
48955,6,2.95,1,2,136,20035,0,0,2011,4,...,1,1,0,0,0,1,0,0,1,1
44966,18,1.25,2,5,140,45537,0,0,2011,4,...,1,1,0,0,0,1,0,0,1,1
13568,3,1.28,1,8,133,21122,1,0,2010,3,...,1,1,0,0,0,1,0,1,0,1
92727,1,5.79,1,1,106,84781,1,0,2011,6,...,1,1,0,0,0,1,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6265,1,5.91,1,9,137,60780,1,0,2010,5,...,1,1,0,0,0,1,0,1,0,1
54886,2,1.66,2,1,147,57354,1,0,2011,5,...,1,1,0,0,0,1,0,1,0,1
76820,3,4.95,1,3,102,37328,0,0,2011,6,...,1,1,0,0,0,1,0,0,1,1


# Saving the intermediate data

In [46]:
df_encoded.to_csv(f"{DATA_OUTPUT_DIR}/{F_1_TRAIN_ENCODED}", sep='|', header=True)
df_test_encoded.to_csv(f"{DATA_OUTPUT_DIR}/{F_1_TEST_ENCODED}", sep='|', header=True)

In [47]:
df_encoded

,Quantity,UnitPrice,randint0,randint1,randint2,randint3,is_CustomerID_null,is_Description_null,InvoiceYear,Num_Words_Desc,...,dumm_StockCode_other,dumm_InvoiceDate_other,dumm_Country_EIRE,dumm_Country_France,dumm_Country_Germany,dumm_Country_United Kingdom,dumm_Country_other,dumm_CustomerID_filled_guest,dumm_CustomerID_filled_other,dumm_Description_filled_other
index,,,,,,,,,,,,,,,,,,,,,
75220,-10,0.00,2,2,105,1964,1,1,2011,1,...,1,1,0,0,0,1,0,1,0,1
48955,6,2.95,1,2,136,20035,0,0,2011,4,...,1,1,0,0,0,1,0,0,1,1
44966,18,1.25,2,5,140,45537,0,0,2011,4,...,1,1,0,0,0,1,0,0,1,1
13568,3,1.28,1,8,133,21122,1,0,2010,3,...,1,1,0,0,0,1,0,1,0,1
92727,1,5.79,1,1,106,84781,1,0,2011,6,...,1,1,0,0,0,1,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6265,1,5.91,1,9,137,60780,1,0,2010,5,...,1,1,0,0,0,1,0,1,0,1
54886,2,1.66,2,1,147,57354,1,0,2011,5,...,1,1,0,0,0,1,0,1,0,1
76820,3,4.95,1,3,102,37328,0,0,2011,6,...,1,1,0,0,0,1,0,0,1,1


In [48]:
df_test_encoded.head()

,Quantity,UnitPrice,randint0,randint1,randint2,randint3,is_CustomerID_null,is_Description_null,InvoiceYear,Num_Words_Desc,...,dumm_StockCode_other,dumm_InvoiceDate_other,dumm_Country_EIRE,dumm_Country_France,dumm_Country_Germany,dumm_Country_United Kingdom,dumm_Country_other,dumm_CustomerID_filled_guest,dumm_CustomerID_filled_other,dumm_Description_filled_other
index,,,,,,,,,,,,,,,,,,,,,
75721,1,1.25,1,6,121,36718,0,0,2011,7,...,1,1,0,0,0,1,0,0,1,1
80184,4,2.95,1,9,141,55893,0,0,2011,4,...,1,1,0,0,0,1,0,0,1,1
19864,2,2.95,1,9,106,94022,0,0,2010,5,...,1,1,0,0,0,1,0,0,1,1
76699,1,7.95,2,5,117,79599,0,0,2011,5,...,1,1,0,0,0,1,0,0,1,1
92991,12,1.25,1,7,122,69365,0,0,2011,7,...,1,1,0,0,0,1,0,0,1,1
